In [1]:
import json
import time

from src.experiments.aux_code import *
from datasets.get_datasets import *
import time
from src.brkga.genetic import genetic as brkga
from src.brkga_variation.genetic import genetic as brkga_var
from src.s_genetic.genetic import *
from src.predicate import *

%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [2]:
source='worksat'
target='companyeconomicsector'

In [3]:
kb = json.loads(open('src/experiments/kb.txt').readline())

In [4]:
kb_source = kb['yago2s']
kb_target = kb['nell_finances']

In [5]:
yago_str = [['worksat(A, B)',
  {'': 'graduatedfrom(C, B), influences(C, D)',
   'true': 'hasacademicadvisor(E, C), hasacademicadvisor(C, F)',
   'true,false': 'iscitizenof(C, G), iscitizenof(A, G)',
   'true,false,false': 'haswebsite(D, H), diedin(C, I)',
   'false': 'hasacademicadvisor(A, J), hasgender(J, K)',
   'false,false': 'graduatedfrom(L, B), livesin(L, M)',
   'false,false,true': 'iscitizenof(A, M), iscitizenof(L, M)',
   'false,false,false': 'hasacademicadvisor(N, A), graduatedfrom(N, P)'},
  {'true,true': [0.388, 273, 1209],
   'true,false,true': [0.37, 69, 352],
   'true,false,false,true': [7.829, 88, 202],
   'true,false,false,false': [18.136, 687, 631],
   'false,true': [0.451, 865, 344],
   'false,false,true,true': [4.969, 33, 98],
   'false,false,true,false': [17.103, 703, 501],
   'false,false,false,true': [3.67, 85, 16],
   'false,false,false,false': [24.03, 1492, 942]}],
 ['worksat(A, B)',
  {'': 'graduatedfrom(C, B), influences(D, C)',
   'true': 'hasacademicadvisor(C, E), influences(F, D)',
   'true,false': 'iscitizenof(C, G), iscitizenof(A, G)',
   'true,false,false': 'iscitizenof(A, H), exports(H, I)',
   'false': 'hasacademicadvisor(J, A), hasgender(A, K)',
   'false,true': 'graduatedfrom(L, B), hasacademicadvisor(M, L)',
   'false,false': 'graduatedfrom(A, B)',
   'false,false,false': 'hasacademicadvisor(A, N), hasacademicadvisor(N, P)'},
  {'true,true': [0.378, 283, 1305],
   'true,false,true': [0.385, 79, 353],
   'true,false,false,true': [11.994, 386, 242],
   'true,false,false,false': [17.15, 545, 655],
   'false,true,true': [0.493, 44, 49],
   'false,true,false': [0.36, 417, 76],
   'false,false,true': [0.272, 0, 95],
   'false,false,false,true': [8.587, 290, 100],
   'false,false,false,false': [29.462, 2251, 1420]}],
 ['worksat(A, B)',
  {'': 'graduatedfrom(C, B), actedin(C, D)',
   'true': 'graduatedfrom(A, B)',
   'true,false': 'wrotemusicfor(E, D), directed(F, D)',
   'false': 'graduatedfrom(G, B), hasacademicadvisor(G, H)',
   'false,true': 'hasacademicadvisor(I, A), graduatedfrom(I, B)',
   'false,true,false': 'livesin(G, J), iscitizenof(A, J)',
   'false,false': 'hasacademicadvisor(K, A), hasgender(A, L)',
   'false,false,false': 'hasacademicadvisor(A, M), hasgender(M, N)'},
  {'true,true': [0.153, 4, 184],
   'true,false,true': [0.414, 433, 1308],
   'true,false,false': [0.49, 77, 67],
   'false,true,true': [0.064, 0, 141],
   'false,true,false,true': [6.93, 67, 220],
   'false,true,false,false': [20.31, 918, 826],
   'false,false,true': [0.362, 427, 79],
   'false,false,false,true': [11.452, 486, 182],
   'false,false,false,false': [27.442, 1883, 1288]}],
 ['worksat(A, B)',
  {'': 'graduatedfrom(C, B), hasacademicadvisor(D, C)',
   'true': 'graduatedfrom(A, B)',
   'true,false': 'owns(B, E), wasbornin(D, F)',
   'true,false,false': 'hasacademicadvisor(G, A), graduatedfrom(G, B)',
   'false': 'graduatedfrom(A, B)',
   'false,false': 'hasacademicadvisor(A, H), hasgender(H, I)',
   'false,false,true': 'hasacademicadvisor(J, A), graduatedfrom(A, K)',
   'false,false,false': 'graduatedfrom(L, B), haschild(M, L)'},
  {'true,true': [0.146, 5, 384],
   'true,false,true': [0.4, 322, 1131],
   'true,false,false,true': [0.635, 0, 98],
   'true,false,false,false': [18.546, 781, 735],
   'false,true': [0.129, 1, 106],
   'false,false,true,true': [5.613, 243, 38],
   'false,false,true,false': [13.303, 627, 258],
   'false,false,false,true': [10.404, 225, 233],
   'false,false,false,false': [28.122, 2091, 1312]}],
 ['worksat(A, B)',
  {'': 'graduatedfrom(C, B), actedin(C, D)',
   'true': 'edited(E, D), directed(E, F)',
   'true,false': 'owns(B, G), wasbornin(C, H)',
   'true,false,true': 'iscitizenof(A, I), islocatedin(D, I)',
   'false': 'graduatedfrom(A, B)',
   'false,false': 'graduatedfrom(A, J)',
   'false,false,true': 'owns(B, K), owns(J, L)',
   'false,false,false': 'graduatedfrom(M, B), isknownfor(M, N)'},
  {'true,true': [0.361, 154, 706],
   'true,false,true,true': [3.41, 16, 66],
   'true,false,true,false': [8.413, 145, 170],
   'true,false,false': [0.397, 199, 617],
   'false,true': [0.118, 2, 306],
   'false,false,true,true': [11.773, 283, 306],
   'false,false,true,false': [26.388, 2214, 1128],
   'false,false,false,true': [3.653, 19, 72],
   'false,false,false,false': [22.516, 1263, 924]}],
 ['worksat(A, B)',
  {'': 'graduatedfrom(A, B)',
   'true': 'hasgender(A, C), owns(B, D)',
   'false': 'graduatedfrom(E, B), actedin(E, F)',
   'false,true': 'haschild(G, E), wasbornin(E, H)',
   'false,true,true': 'livesin(I, H), hasgender(A, J)',
   'false,true,false': 'wrotemusicfor(K, F), ismarriedto(K, L)',
   'false,false': 'hasacademicadvisor(M, A), graduatedfrom(M, B)',
   'false,false,false': 'hasacademicadvisor(N, A), graduatedfrom(N, P)'},
  {'true,true': [0.163, 4, 183],
   'true,false': [0.111, 2, 307],
   'false,true,true,true': [5.377, 36, 271],
   'false,true,true,false': [1.339, 5, 4],
   'false,true,false,true': [12.385, 220, 730],
   'false,true,false,false': [11.418, 249, 370],
   'false,false,true': [0.096, 0, 95],
   'false,false,false,true': [8.864, 426, 113],
   'false,false,false,false': [35.62, 3353, 2222]}],
 ['worksat(A, B)',
  {'': 'graduatedfrom(C, B), influences(D, C)',
   'true': 'livesin(D, E), hasacademicadvisor(D, F)',
   'true,false': 'iscitizenof(C, G), iscitizenof(A, G)',
   'true,false,false': 'iscitizenof(A, H), exports(H, I)',
   'false': 'hasacademicadvisor(A, J), hasacademicadvisor(J, K)',
   'false,false': 'graduatedfrom(A, L), owns(L, M)',
   'false,false,true': 'hasacademicadvisor(N, A), iscitizenof(A, P)',
   'false,false,false': 'influences(A, Q), graduatedfrom(R, B)'},
  {'true,true': [0.345, 223, 1101],
   'true,false,true': [0.367, 90, 443],
   'true,false,false,true': [11.432, 399, 242],
   'true,false,false,false': [17.013, 581, 769],
   'false,true': [0.4, 500, 155],
   'false,false,true,true': [2.32, 41, 6],
   'false,false,true,false': [16.611, 802, 459],
   'false,false,false,true': [2.891, 47, 13],
   'false,false,false,false': [24.898, 1612, 1107]}],
 ['worksat(A, B)',
  {'': 'hasacademicadvisor(C, A), graduatedfrom(C, B)',
   'false': 'graduatedfrom(D, B), wrotemusicfor(D, E)',
   'false,true': 'directed(F, E), islocatedin(E, G)',
   'false,true,true': 'diedin(A, H), hascurrency(G, I)',
   'false,true,false': 'hasacademicadvisor(A, J), haswebsite(D, K)',
   'false,false': 'graduatedfrom(L, B), haschild(L, M)',
   'false,false,true': 'iscitizenof(L, N), iscitizenof(A, N)',
   'false,false,false': 'graduatedfrom(A, B)'},
  {'true': [0.085, 0, 241],
   'false,true,true,true': [2.65, 13, 35],
   'false,true,true,false': [6.432, 51, 415],
   'false,true,false,true': [2.016, 5, 50],
   'false,true,false,false': [6.758, 75, 167],
   'false,false,true,true': [4.937, 28, 239],
   'false,false,true,false': [19.173, 777, 1057],
   'false,false,false,true': [1.365, 1, 161],
   'false,false,false,false': [34.069, 3345, 1930]}],
 ['worksat(A, B)',
  {'': 'graduatedfrom(A, B)',
   'false': 'graduatedfrom(C, B), haschild(D, C)',
   'false,true': 'iscitizenof(C, E), iscitizenof(A, E)',
   'false,true,true': 'hascurrency(E, F), ismarriedto(G, D)',
   'false,true,false': 'iscitizenof(D, H), hasgender(A, I)',
   'false,false': 'graduatedfrom(A, J), owns(J, K)',
   'false,false,true': 'owns(B, L)',
   'false,false,false': 'graduatedfrom(M, B), owns(B, N)'},
  {'true': [0.13, 6, 490],
   'false,true,true,true': [3.882, 19, 112],
   'false,true,true,false': [5.293, 35, 297],
   'false,true,false,true': [9.176, 133, 478],
   'false,true,false,false': [17.117, 624, 748],
   'false,false,true,true': [10.951, 245, 269],
   'false,false,true,false': [15.82, 927, 360],
   'false,false,false,true': [11.022, 334, 241],
   'false,false,false,false': [26.87, 1972, 1300]}],
 ['worksat(A, B)',
  {'': 'graduatedfrom(A, B)',
   'false': 'hasacademicadvisor(A, C), graduatedfrom(D, B)',
   'false,true': 'iscitizenof(D, E), iscitizenof(A, E)',
   'false,true,true': 'dealswith(E, F)',
   'false,true,false': 'hasacademicadvisor(D, G), hasacademicadvisor(G, H)',
   'false,false': 'graduatedfrom(I, B), isknownfor(I, J)',
   'false,false,true': 'influences(K, I), influences(L, A)',
   'false,false,false': 'graduatedfrom(A, M)'},
  {'true': [0.126, 6, 490],
   'false,true,true,true': [3.795, 10, 149],
   'false,true,true,false': [6.43, 75, 199],
   'false,true,false,true': [10.456, 216, 395],
   'false,true,false,false': [8.926, 398, 107],
   'false,false,true,true': [1.073, 1, 39],
   'false,false,true,false': [7.528, 83, 313],
   'false,false,false,true': [27.948, 2231, 1502],
   'false,false,false,false': [23.126, 1275, 1101]}]]

In [6]:
pred_target = create_pred_target(kb_target)


In [1]:
import copy
src_struct = copy.deepcopy(yago_str)
new_src_struct = []
for i in range(0, len(src_struct)):
    new_src_struct.append(define_individual(src_struct[i], i))  
structured_src = src_struct


# target = f'recursion_{target}'


NameError: name 'yago_str' is not defined

In [8]:
finances_dataset = datasets.load('nell_finances', kb_target, target=target, seed=441773, balanced=0)
train_facts, train_pos, train_neg, test_facts, test_pos, test_neg = get_train_test(finances_dataset, 0)

0


In [9]:
facts = []
facts.extend(train_facts[0])
facts.extend(train_facts[1])
facts.extend(test_facts)
train_facts = [copy.deepcopy(facts), copy.deepcopy(facts)]
test_facts = [copy.deepcopy(facts)]

# Crossover original

## sem revisão

In [10]:
start = time.time()
res = genetic(new_src_struct, target, source, train_pos, 
                                         train_neg, train_facts, kb_source, kb_target, pred_target,
                                          NUM_GEN=14, pop_size=1)
print("FIM DO GROOT GENETIC: ", time.time()-start)

GENERATION:  0
MELHOR RESULTADO:  -0.9313525


Process ForkPoolWorker-39:
Process ForkPoolWorker-37:
Process ForkPoolWorker-36:
Process ForkPoolWorker-40:
Process ForkPoolWorker-35:
Process ForkPoolWorker-38:
Process ForkPoolWorker-34:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):


In [ ]:
res

In [ ]:
def get_k_best_individuals(population, k):
    best_ind = []
    all_fitness = []
    for i in range(0, len(population)):
        all_fitness.append((population[i].fitness.values[0], i))
    
    sorted_ind = sorted(all_fitness, reverse=False, key=lambda tup: tup[0])
    for i in range(0, k):
        best_ind.append(population[sorted_ind[i][1]])
    return best_ind

In [ ]:
# final_results[f'groot_test:{source}->{target}'] = []
# final_results[f'groot_test_rev:{source}->{target}'] = []
# final_results[f'inf_res:{source}->{target}'] = []
# final_results[f'inf_res_rev:{source}->{target}'] = []

# uwcse_dataset = datasets.load('uwcse', kb_target, target=target, seed=441773)
# train_facts, train_pos, train_neg, test_facts, test_pos, test_neg = get_train_test(uwcse_dataset, 1)
# tr_train_pos = []
# tr_train_neg = []
# tr_train_facts = []

# for i in range(0, len(train_pos)):
#     tr_train_pos.extend(train_pos[i])
#     tr_train_neg.extend(train_neg[i])
#     tr_train_facts.extend(train_facts[i])

# train = [tr_train_pos, tr_train_neg, tr_train_facts]
test = [test_pos, test_neg, test_facts[0]]

individuals = get_k_best_individuals(res[0].population, 3)

n_ind = 1
for individual in individuals:
    print("INDIVIDUO ", n_ind)
    refine, transfer = get_refine_transfer(individual, source, target, 'yago2s', 'nell_finances')
    for i in range(len(train_pos)):
        ttrain = [train_pos[i], train_neg[i][:len(train_pos[i])], train_facts[i]]
        res_ = test_refine_transfer(kb_target, target, refine, transfer, ttrain, test)
        print(res_)
#         final_results[f'groot_test:{source}->{target}'].append(res)
#         import os
    #     thisFile = f'boostsrl/best/test/results_{target.replace('recursion_', '')}.db'
    #     thisFile = f'boostsrl/best/test/results_{target}.db'
    #     base = os.path.splitext(thisFile)[0]
    #     os.rename(thisFile, base + ".txt")
    #     tt = open('boostsrl/best/test/results_advisedby.txt', 'r').readlines()
    #     final = []
    #     for i in tt:
    #         final.append(i.replace('\n', ''))
    #     final_results[f'inf_res:{source}->{target}'].append(final)
        print("===========================================================================")
#         break

    n_ind += 1

## Aleatório

In [ ]:
start = time.time()
res_a_c = genetic(new_src_struct, target, source, train_pos, 
                                         train_neg, train_facts, kb_source, kb_target, pred_target,
                                          NUM_GEN=14, pop_size=30, revision='random')
print("FIM DO GROOT GENETIC: ", time.time()-start)

In [ ]:
res_a_c

In [ ]:
def get_k_best_individuals(population, k):
    best_ind = []
    all_fitness = []
    for i in range(0, len(population)):
        all_fitness.append((population[i].fitness.values[0], i))
    
    sorted_ind = sorted(all_fitness, reverse=True, key=lambda tup: tup[0])
    for i in range(0, k):
        best_ind.append(population[sorted_ind[i][1]])
    return best_ind

In [ ]:
final_results = {}
final_results[f'{source}->{target}'] = []
final_results[f'{source}->{target}'].append(res)
best = final_results[f'{source}->{target}'][0]
for i in final_results[f'{source}->{target}']:
    if i[1][-1] > best[1][-1]:
        best = i

In [ ]:
# final_results[f'groot_test:{source}->{target}'] = []
# final_results[f'groot_test_rev:{source}->{target}'] = []
# final_results[f'inf_res:{source}->{target}'] = []
# final_results[f'inf_res_rev:{source}->{target}'] = []

# uwcse_dataset = datasets.load('uwcse', kb_target, target=target, seed=441773)
# train_facts, train_pos, train_neg, test_facts, test_pos, test_neg = get_train_test(uwcse_dataset, 1)
# tr_train_pos = []
# tr_train_neg = []
# tr_train_facts = []

# for i in range(0, len(train_pos)):
#     tr_train_pos.extend(train_pos[i])
#     tr_train_neg.extend(train_neg[i])
#     tr_train_facts.extend(train_facts[i])

# train = [tr_train_pos, tr_train_neg, tr_train_facts]
test = [test_pos, test_neg, test_facts[0]]

individuals = get_k_best_individuals(res_a_c[0].population, 3)

n_ind = 1
for individual in individuals:
    print("INDIVIDUO ", n_ind)
    refine, transfer = get_refine_transfer(individual, source, target, 'yago2s', 'nell_finances')
    for i in range(len(train_pos)):
        ttrain = [train_pos[i], train_neg[i][:len(train_pos[i])], train_facts[i]]
        res_ = test_refine_transfer(kb_target, target, refine, transfer, ttrain, test)
        print(res_)
#         final_results[f'groot_test:{source}->{target}'].append(res)
#         import os
    #     thisFile = f'boostsrl/best/test/results_{target.replace('recursion_', '')}.db'
    #     thisFile = f'boostsrl/best/test/results_{target}.db'
    #     base = os.path.splitext(thisFile)[0]
    #     os.rename(thisFile, base + ".txt")
    #     tt = open('boostsrl/best/test/results_advisedby.txt', 'r').readlines()
    #     final = []
    #     for i in tt:
    #         final.append(i.replace('\n', ''))
    #     final_results[f'inf_res:{source}->{target}'].append(final)
        print("===========================================================================")
#         break

    n_ind += 1

## Guiado

In [ ]:
start = time.time()
res_g_c = genetic(new_src_struct, target, source, train_pos, 
                                         train_neg, train_facts, kb_source, kb_target, pred_target,
                                          NUM_GEN=14, pop_size=30, revision='guided')
print("FIM DO GROOT GENETIC: ", time.time()-start)

In [ ]:
res_g_c

In [ ]:
# final_results[f'groot_test:{source}->{target}'] = []
# final_results[f'groot_test_rev:{source}->{target}'] = []
# final_results[f'inf_res:{source}->{target}'] = []
# final_results[f'inf_res_rev:{source}->{target}'] = []

# uwcse_dataset = datasets.load('uwcse', kb_target, target=target, seed=441773)
# train_facts, train_pos, train_neg, test_facts, test_pos, test_neg = get_train_test(uwcse_dataset, 1)
# tr_train_pos = []
# tr_train_neg = []
# tr_train_facts = []

# for i in range(0, len(train_pos)):
#     tr_train_pos.extend(train_pos[i])
#     tr_train_neg.extend(train_neg[i])
#     tr_train_facts.extend(train_facts[i])

# train = [tr_train_pos, tr_train_neg, tr_train_facts]
test = [test_pos, test_neg, test_facts[0]]

individuals = get_k_best_individuals(res_g_c[0].population, 3)

n_ind = 1
for individual in individuals:
    print("INDIVIDUO ", n_ind)
    refine, transfer = get_refine_transfer(individual, source, target, 'yago2s', 'nell_finances')
    for i in range(len(train_pos)):
        ttrain = [train_pos[i], train_neg[i][:len(train_pos[i])], train_facts[i]]
        res_ = test_refine_transfer(kb_target, target, refine, transfer, ttrain, test)
        print(res_)
#         final_results[f'groot_test:{source}->{target}'].append(res)
#         import os
    #     thisFile = f'boostsrl/best/test/results_{target.replace('recursion_', '')}.db'
    #     thisFile = f'boostsrl/best/test/results_{target}.db'
    #     base = os.path.splitext(thisFile)[0]
    #     os.rename(thisFile, base + ".txt")
    #     tt = open('boostsrl/best/test/results_advisedby.txt', 'r').readlines()
    #     final = []
    #     for i in tt:
    #         final.append(i.replace('\n', ''))
    #     final_results[f'inf_res:{source}->{target}'].append(final)
        print("===========================================================================")
#         break

    n_ind += 1

## Mixed

In [ ]:
start = time.time()
res_m_c = genetic(new_src_struct, target, source, train_pos, 
                                         train_neg, train_facts, kb_source, kb_target, pred_target,
                                          NUM_GEN=14, pop_size=30, revision='mixed')
print("FIM DO GROOT GENETIC: ", time.time()-start)

In [ ]:
res_m_c

In [ ]:
# final_results[f'groot_test:{source}->{target}'] = []
# final_results[f'groot_test_rev:{source}->{target}'] = []
# final_results[f'inf_res:{source}->{target}'] = []
# final_results[f'inf_res_rev:{source}->{target}'] = []

# uwcse_dataset = datasets.load('uwcse', kb_target, target=target, seed=441773)
# train_facts, train_pos, train_neg, test_facts, test_pos, test_neg = get_train_test(uwcse_dataset, 1)
# tr_train_pos = []
# tr_train_neg = []
# tr_train_facts = []

# for i in range(0, len(train_pos)):
#     tr_train_pos.extend(train_pos[i])
#     tr_train_neg.extend(train_neg[i])
#     tr_train_facts.extend(train_facts[i])

# train = [tr_train_pos, tr_train_neg, tr_train_facts]
test = [test_pos, test_neg, test_facts[0]]

individuals = get_k_best_individuals(res_m_c[0].population, 3)

n_ind = 1
for individual in individuals:
    print("INDIVIDUO ", n_ind)
    refine, transfer = get_refine_transfer(individual, source, target, 'yago2s', 'nell_finances')
    for i in range(len(train_pos)):
        ttrain = [train_pos[i], train_neg[i][:len(train_pos[i])], train_facts[i]]
        res_ = test_refine_transfer(kb_target, target, refine, transfer, ttrain, test)
        print(res_)
#         final_results[f'groot_test:{source}->{target}'].append(res)
#         import os
    #     thisFile = f'boostsrl/best/test/results_{target.replace('recursion_', '')}.db'
    #     thisFile = f'boostsrl/best/test/results_{target}.db'
    #     base = os.path.splitext(thisFile)[0]
    #     os.rename(thisFile, base + ".txt")
    #     tt = open('boostsrl/best/test/results_advisedby.txt', 'r').readlines()
    #     final = []
    #     for i in tt:
    #         final.append(i.replace('\n', ''))
    #     final_results[f'inf_res:{source}->{target}'].append(final)
        print("===========================================================================")
#         break

    n_ind += 1

# Crossover modificado

## sem revisão

In [ ]:
start = time.time()
res_r_m = genetic(new_src_struct, target, source, train_pos, 
                                         train_neg, train_facts, kb_source, kb_target, pred_target,
                                          NUM_GEN=14, pop_size=30)
print("FIM DO GROOT GENETIC: ", time.time()-start)

In [ ]:
res_r_m

In [ ]:
# final_results[f'groot_test:{source}->{target}'] = []
# final_results[f'groot_test_rev:{source}->{target}'] = []
# final_results[f'inf_res:{source}->{target}'] = []
# final_results[f'inf_res_rev:{source}->{target}'] = []

# uwcse_dataset = datasets.load('uwcse', kb_target, target=target, seed=441773)
# train_facts, train_pos, train_neg, test_facts, test_pos, test_neg = get_train_test(uwcse_dataset, 1)
# tr_train_pos = []
# tr_train_neg = []
# tr_train_facts = []

# for i in range(0, len(train_pos)):
#     tr_train_pos.extend(train_pos[i])
#     tr_train_neg.extend(train_neg[i])
#     tr_train_facts.extend(train_facts[i])

# train = [tr_train_pos, tr_train_neg, tr_train_facts]
test = [test_pos, test_neg, test_facts]

individuals = get_k_best_individuals(res_r_m[0].population, 3)

n_ind = 1
for individual in individuals:
    print("INDIVIDUO ", n_ind)
    refine, transfer = get_refine_transfer(individual, source, target, 'imdb', 'uwcse')
    for i in range(len(train_pos)):
        ttrain = [train_pos[i], train_neg[i][:len(train_pos[i])], train_facts[i]]
        res_ = test_refine_transfer(kb_target, target, refine, transfer, ttrain, test)
        print(res_)
#         final_results[f'groot_test:{source}->{target}'].append(res)
#         import os
    #     thisFile = f'boostsrl/best/test/results_{target.replace('recursion_', '')}.db'
    #     thisFile = f'boostsrl/best/test/results_{target}.db'
    #     base = os.path.splitext(thisFile)[0]
    #     os.rename(thisFile, base + ".txt")
    #     tt = open('boostsrl/best/test/results_advisedby.txt', 'r').readlines()
    #     final = []
    #     for i in tt:
    #         final.append(i.replace('\n', ''))
    #     final_results[f'inf_res:{source}->{target}'].append(final)
        print("===========================================================================")
#         break

    n_ind += 1

## Aleatório

In [ ]:
start = time.time()
res_a_m = genetic(new_src_struct, target, source, train_pos, 
                                         train_neg, train_facts, kb_source, kb_target, pred_target,
                                          NUM_GEN=14, pop_size=30, revision='random')
print("FIM DO GROOT GENETIC: ", time.time()-start)

In [ ]:
res_a_m

In [ ]:
# final_results[f'groot_test:{source}->{target}'] = []
# final_results[f'groot_test_rev:{source}->{target}'] = []
# final_results[f'inf_res:{source}->{target}'] = []
# final_results[f'inf_res_rev:{source}->{target}'] = []

# uwcse_dataset = datasets.load('uwcse', kb_target, target=target, seed=441773)
# train_facts, train_pos, train_neg, test_facts, test_pos, test_neg = get_train_test(uwcse_dataset, 1)
# tr_train_pos = []
# tr_train_neg = []
# tr_train_facts = []

# for i in range(0, len(train_pos)):
#     tr_train_pos.extend(train_pos[i])
#     tr_train_neg.extend(train_neg[i])
#     tr_train_facts.extend(train_facts[i])

# train = [tr_train_pos, tr_train_neg, tr_train_facts]
test = [test_pos, test_neg, test_facts]

individuals = get_k_best_individuals(res_a_m[0].population, 3)

n_ind = 1
for individual in individuals:
    print("INDIVIDUO ", n_ind)
    refine, transfer = get_refine_transfer(individual, source, target, 'imdb', 'uwcse')
    for i in range(len(train_pos)):
        ttrain = [train_pos[i], train_neg[i][:len(train_pos[i])], train_facts[i]]
        res_ = test_refine_transfer(kb_target, target, refine, transfer, ttrain, test)
        print(res_)
#         final_results[f'groot_test:{source}->{target}'].append(res)
#         import os
    #     thisFile = f'boostsrl/best/test/results_{target.replace('recursion_', '')}.db'
    #     thisFile = f'boostsrl/best/test/results_{target}.db'
    #     base = os.path.splitext(thisFile)[0]
    #     os.rename(thisFile, base + ".txt")
    #     tt = open('boostsrl/best/test/results_advisedby.txt', 'r').readlines()
    #     final = []
    #     for i in tt:
    #         final.append(i.replace('\n', ''))
    #     final_results[f'inf_res:{source}->{target}'].append(final)
        print("===========================================================================")
#         break

    n_ind += 1

## Guiado

In [ ]:
start = time.time()
res_g_m = genetic(new_src_struct, target, source, train_pos, 
                                         train_neg, train_facts, kb_source, kb_target, pred_target,
                                          NUM_GEN=14, pop_size=30, revision='guided')
print("FIM DO GROOT GENETIC: ", time.time()-start)

In [ ]:
res_g_m

In [ ]:
# final_results[f'groot_test:{source}->{target}'] = []
# final_results[f'groot_test_rev:{source}->{target}'] = []
# final_results[f'inf_res:{source}->{target}'] = []
# final_results[f'inf_res_rev:{source}->{target}'] = []

# uwcse_dataset = datasets.load('uwcse', kb_target, target=target, seed=441773)
# train_facts, train_pos, train_neg, test_facts, test_pos, test_neg = get_train_test(uwcse_dataset, 1)
# tr_train_pos = []
# tr_train_neg = []
# tr_train_facts = []

# for i in range(0, len(train_pos)):
#     tr_train_pos.extend(train_pos[i])
#     tr_train_neg.extend(train_neg[i])
#     tr_train_facts.extend(train_facts[i])

# train = [tr_train_pos, tr_train_neg, tr_train_facts]
test = [test_pos, test_neg, test_facts]

individuals = get_k_best_individuals(res_g_m[0].population, 3)

n_ind = 1
for individual in individuals:
    print("INDIVIDUO ", n_ind)
    refine, transfer = get_refine_transfer(individual, source, target, 'imdb', 'uwcse')
    for i in range(len(train_pos)):
        ttrain = [train_pos[i], train_neg[i][:len(train_pos[i])], train_facts[i]]
        res_ = test_refine_transfer(kb_target, target, refine, transfer, ttrain, test)
        print(res_)
#         final_results[f'groot_test:{source}->{target}'].append(res)
#         import os
    #     thisFile = f'boostsrl/best/test/results_{target.replace('recursion_', '')}.db'
    #     thisFile = f'boostsrl/best/test/results_{target}.db'
    #     base = os.path.splitext(thisFile)[0]
    #     os.rename(thisFile, base + ".txt")
    #     tt = open('boostsrl/best/test/results_advisedby.txt', 'r').readlines()
    #     final = []
    #     for i in tt:
    #         final.append(i.replace('\n', ''))
    #     final_results[f'inf_res:{source}->{target}'].append(final)
        print("===========================================================================")
#         break

    n_ind += 1

## Mixed

In [ ]:
start = time.time()
res_m_m = genetic(new_src_struct, target, source, train_pos, 
                                         train_neg, train_facts, kb_source, kb_target, pred_target,
                                          NUM_GEN=14, pop_size=30, revision='mixed')
print("FIM DO GROOT GENETIC: ", time.time()-start)

In [ ]:
res_m_m

In [ ]:
# final_results[f'groot_test:{source}->{target}'] = []
# final_results[f'groot_test_rev:{source}->{target}'] = []
# final_results[f'inf_res:{source}->{target}'] = []
# final_results[f'inf_res_rev:{source}->{target}'] = []

# uwcse_dataset = datasets.load('uwcse', kb_target, target=target, seed=441773)
# train_facts, train_pos, train_neg, test_facts, test_pos, test_neg = get_train_test(uwcse_dataset, 1)
# tr_train_pos = []
# tr_train_neg = []
# tr_train_facts = []

# for i in range(0, len(train_pos)):
#     tr_train_pos.extend(train_pos[i])
#     tr_train_neg.extend(train_neg[i])
#     tr_train_facts.extend(train_facts[i])

# train = [tr_train_pos, tr_train_neg, tr_train_facts]
test = [test_pos, test_neg, test_facts]

individuals = get_k_best_individuals(res_m_m[0].population, 3)

n_ind = 1
for individual in individuals:
    print("INDIVIDUO ", n_ind)
    refine, transfer = get_refine_transfer(individual, source, target, 'imdb', 'uwcse')
    for i in range(len(train_pos)):
        ttrain = [train_pos[i], train_neg[i][:len(train_pos[i])], train_facts[i]]
        res_ = test_refine_transfer(kb_target, target, refine, transfer, ttrain, test)
        print(res_)
#         final_results[f'groot_test:{source}->{target}'].append(res)
#         import os
    #     thisFile = f'boostsrl/best/test/results_{target.replace('recursion_', '')}.db'
    #     thisFile = f'boostsrl/best/test/results_{target}.db'
    #     base = os.path.splitext(thisFile)[0]
    #     os.rename(thisFile, base + ".txt")
    #     tt = open('boostsrl/best/test/results_advisedby.txt', 'r').readlines()
    #     final = []
    #     for i in tt:
    #         final.append(i.replace('\n', ''))
    #     final_results[f'inf_res:{source}->{target}'].append(final)
        print("===========================================================================")
#         break

    n_ind += 1

In [25]:
train = [tr_train_pos, tr_train_neg, tr_train_facts]
test = [test_pos, test_neg, test_facts[0]]
aa = test_tree_b(source, target, kb_source, kb_target, structured_src, train, test)
aa

******************************************
Performing Parameter Learning
******************************************
Refine
['worksat(A, B)', {'': 'graduatedfrom(C, B), influences(C, D)', 'true': 'hasacademicadvisor(E, C), hasacademicadvisor(C, F)', 'true,false': 'iscitizenof(C, G), iscitizenof(A, G)', 'true,false,false': 'haswebsite(D, H), diedin(C, I)', 'false': 'hasacademicadvisor(A, J), hasgender(J, K)', 'false,false': 'graduatedfrom(L, B), livesin(L, M)', 'false,false,true': 'iscitizenof(A, M), iscitizenof(L, M)', 'false,false,false': 'hasacademicadvisor(N, A), graduatedfrom(N, P)'}, {'true,true': [0.388, 273, 1209], 'true,false,true': [0.37, 69, 352], 'true,false,false,true': [7.829, 88, 202], 'true,false,false,false': [18.136, 687, 631], 'false,true': [0.451, 865, 344], 'false,false,true,true': [4.969, 33, 98], 'false,false,true,false': [17.103, 703, 501], 'false,false,false,true': [3.67, 85, 16], 'false,false,false,false': [24.03, 1492, 942]}]
['worksat(A, B)', {'': 'graduatedfr

WILL Produced-Tree #1
% FOR companyeconomicsector(A, B):
%   if ( economicsectorcompany(B, C), companyalsoknownas(C, D) )
%   then if ( acquired(E, C), acquired(C, F) )
%   | then return 0.8022483139814973;  // std dev = 2,915, 161,000 (wgt'ed) examples reached here.  /* #neg=9 #pos=152 */
%   | else if ( companyceo(C, G), companyceo(A, G) )
%   | | then return 0.8581489350995123;  // std dev = 0,000, 18,000 (wgt'ed) examples reached here.  /* #pos=18 */
%   | | else if ( countryhascompanyoffice(H, D), bankchiefexecutiveceo(C, I) )
%   | | | then return 0;  // std dev = 0,000, 0,000 (wgt'ed) examples reached here.
%   | | | else return 0.7519542448340228;  // std dev = 5,672, 339,000 (wgt'ed) examples reached here.  /* #neg=36 #pos=303 */
%   else if ( acquired(A, J), bankboughtbank(J, K) )
%   | then return 0.19148226843284552;  // std dev = 0,816, 3,000 (wgt'ed) examples reached here.  /* #neg=2 #pos=1 */
%   | else if ( economicsectorcompany(B, L) )
%   | | then if ( companyceo(A, M

Results
   AUC ROC   = 1.0
   AUC PR    = 1.0
   CLL	      = -0.076521
   Precision = 1.0 at threshold = 0.5
   Recall    = 1.0
   F1        = 1.0


Total learning time: 14.405 seconds
Total inference time: 29.894 seconds
AUC ROC: 1.0




Exception: ('Encountered problems while running process: ', '(cd boostsrl; java -jar v1-0.jar -i -model train/models/ -test test/ -target companyeconomicsector -trees 10 -aucJarPath . > test_output.txt 2>&1)')

In [24]:
with open('teste.txt', 'w') as f:
    for line in test_facts[0]:
        print(line)
        f.write(line + '\n')

['cityhascompanyoffice(newyork,aolvideo).', 'bankbankincountry(national,swaziland).', 'acquired(sapag,businessobjectssa).', 'cityhascompanyoffice(cincinnati,wcpo).', 'bankbankincountry(fortis,belgium).', 'cityhascompanyoffice(anchorage,eraaviation).', 'acquired(google,dejanews).', 'cityhascompanyoffice(tucsonarizona,nissan).', 'cityhascompanyoffice(fresno,kgpe).', 'cityhascompanyoffice(washingtondc,imf).', 'countryhascompanyoffice(america,kia).', 'economicsectorcompany(investment,deanwitter).', 'cityhascompanyoffice(honolulu,acura).', 'cityhascompanyoffice(omaha,qwest).', 'cityhascompanyoffice(blanchard,national).', 'acquired(gm,opel).', 'acquired(nbcuniversal,telemundo).', 'cityhascompanyoffice(seattle,national).', 'companyceo(cypresssemiconductor,tjrodgers).', 'countryhascompanyoffice(america,chevrolet).', 'cityhascompanyoffice(frankfurt,ryanair).', 'cityhascompanyoffice(newyork,metlife).', 'companyalsoknownas(samsung,philipselectronics).', 'cityhascompanyoffice(newcastle,ryanair).',

TypeError: can only concatenate list (not "str") to list

In [11]:
tr_train_pos = []
tr_train_neg = []
tr_train_facts = []

for i in range(0, len(train_pos)):
    tr_train_pos.extend(train_pos[i])
    tr_train_neg.extend(train_neg[i])
    tr_train_facts.extend(train_facts[i])

In [10]:
background_knowledge_src = boostsrl.modes(kb_target, [target], useStdLogicVariables=False, 
                                      maxTreeDepth=3, nodeSize=2, numOfClauses=8)
sys.path.insert(0, '../TreeBoostler')

from revision import revision

def print_function(message):
        global experiment_title
        global nbr
        experiment_title = f'Twitter->Yeast2: {source}->{target}'
        nbr = 1
        if not os.path.exists('experiments/' + experiment_title):
            os.makedirs('experiments/' + experiment_title)
        with open('experiments/' + experiment_title + '/' + str(nbr) + '_' + experiment_title + '.txt', 'a') as f:
            print(message, file=f)
            print(message)

model = revision.learn_test_model(background_knowledge_src, boostsrl, target, tr_train_pos, tr_train_neg, copy.deepcopy(facts), test_pos, test_neg, copy.deepcopy(facts), trees=10, print_function=print_function)

NameError: name 'tr_train_pos' is not defined

In [12]:
model